In [1]:
%pip install pandas numpy matplotlib plotly networkx pyDatalog

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Boilerplate for AI Assignment — Knowledge Representation, Reasoning and Planning
# CSE 643

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
from pyDatalog import pyDatalog
from collections import defaultdict, deque

## ****IMPORTANT****
## Don't import or use any other libraries other than defined above
## Otherwise your code file will be rejected in the automated testing

In [3]:
# ------------------ Global Variables ------------------
route_to_stops = defaultdict(list)  # Mapping of route IDs to lists of stops
trip_to_route = {}                   # Mapping of trip IDs to route IDs
stop_trip_count = defaultdict(int)    # Count of trips for each stop
fare_rules = {}                      # Mapping of route IDs to fare information
merged_fare_df = None                # To be initialized in create_kb()

# Load static data from GTFS (General Transit Feed Specification) files
df_stops = pd.read_csv('GTFS/stops.txt')
df_routes = pd.read_csv('GTFS/routes.txt')
df_stop_times = pd.read_csv('GTFS/stop_times.txt')
df_fare_attributes = pd.read_csv('GTFS/fare_attributes.txt')
df_trips = pd.read_csv('GTFS/trips.txt')
df_fare_rules = pd.read_csv('GTFS/fare_rules.txt')

In [4]:
# ------------------ Function Definitions ------------------

# Function to create knowledge base from the loaded data
def create_kb():
    """
    Create knowledge base by populating global variables with information from loaded datasets.
    It establishes the relationships between routes, trips, stops, and fare rules.
    
    Returns:
        None
    """
    global route_to_stops, trip_to_route, stop_trip_count, fare_rules, merged_fare_df

    # Create trip_id to route_id mapping
    for tmp, row in df_trips.iterrows():
        trip_to_route[row['trip_id']] = row['route_id']

    # Map route_id to a list of stops in order of their sequence
    for tmp, row in df_stop_times.iterrows():
        route_id = trip_to_route.get(row['trip_id'])
        if route_id:
            # Only add (sequence, stop_id) if it doesn't already exist for the route
            route_to_stops[route_id].append((row['stop_sequence'], row['stop_id']))
            # Count trips per stop
            stop_trip_count[row['stop_id']] += 1

    # Process each route to retain only unique stop IDs in order
    for route_id, stops in route_to_stops.items():
        # Filter and sort based on sequence, then extract stop_ids
        unique_stops = sorted(set(stops), key=lambda x: x[0])
        route_to_stops[route_id] = [stop_id for _, stop_id in unique_stops]

    # Create fare rules for routes
    fare_rules = df_fare_rules.set_index('route_id').T.to_dict()

    # Merge fare rules and attributes into a single DataFrame
    merged_fare_df = pd.merge(df_fare_rules, df_fare_attributes, on='fare_id', how='inner')

In [5]:
# Function to find the top 5 busiest routes based on the number of trips
def get_busiest_routes():
    """
    Identify the top 5 busiest routes based on trip counts.

    Returns:
        list: A list of tuples, where each tuple contains:
              - route_id (int): The ID of the route.
              - trip_count (int): The number of trips for that route.
    """
    route_trip_count = defaultdict(int)

    for trip_id, route_id in trip_to_route.items():
        route_trip_count[route_id] += 1

    busiest_routes = sorted(route_trip_count.items(), key=lambda x: x[1], reverse=True)[:5]

    return busiest_routes

# Function to find the top 5 stops with the most frequent trips
def get_most_frequent_stops():
    """
    Identify the top 5 stops with the highest number of trips.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (int): The ID of the stop.
              - trip_count (int): The number of trips for that stop.
    """
    most_frequent_stops = sorted(stop_trip_count.items(), key=lambda x: x[1], reverse=True)[:5]

    return most_frequent_stops

# Function to find the top 5 busiest stops based on the number of routes passing through them
def get_top_5_busiest_stops():
    """
    Identify the top 5 stops with the highest number of different routes.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (int): The ID of the stop.
              - route_count (int): The number of routes passing through that stop.
    """
    stop_to_routes = defaultdict(set)

    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            stop_to_routes[stop_id].add(route_id)

    stop_route_count = {stop_id: len(routes) for stop_id, routes in stop_to_routes.items()}

    top_5_busiest_stops = sorted(stop_route_count.items(), key=lambda x: x[1], reverse=True)[:5]

    return top_5_busiest_stops

# Function to identify the top 5 pairs of stops with only one direct route between them
def get_stops_with_one_direct_route():
    """
    Identify the top 5 pairs of consecutive stops (start and end) connected by exactly one direct route. 
    The pairs are sorted by the combined frequency of trips passing through both stops.

    Returns:
        list: A list of tuples, where each tuple contains:
              - pair (tuple): A tuple with two stop IDs (stop_1, stop_2).
              - route_id (int): The ID of the route connecting the two stops.
    """
    stop_pairs = defaultdict(lambda: defaultdict(int))

    for route_id, stops in route_to_stops.items():
        for i in range(len(stops) - 1):
            pair = (stops[i], stops[i + 1])
            stop_pairs[pair][route_id] += 1
    single_route_pairs = [(pair, route_id) for pair, routes in stop_pairs.items() if len(routes) == 1 for route_id in routes]
    sorted_pairs = sorted(single_route_pairs, key=lambda x: stop_trip_count[x[0][0]] + stop_trip_count[x[0][1]], reverse=True)[:5]
    return sorted_pairs

# Function to get merged fare DataFrame
# No need to change this function
def get_merged_fare_df():
    """
    Retrieve the merged fare DataFrame.

    Returns:
        DataFrame: The merged fare DataFrame containing fare rules and attributes.
    """
    global merged_fare_df
    return merged_fare_df

# Visualize the stop-route graph interactively
def visualize_stop_route_graph_interactive(route_to_stops):
    """
    Visualize the stop-route graph using Plotly for interactive exploration.

    Args:
        route_to_stops (dict): A dictionary mapping route IDs to lists of stops.

    Returns:
        None
    """
    G = nx.Graph()

    for route_id, stops in route_to_stops.items():
        for i in range(len(stops) - 1):
            G.add_edge(stops[i], stops[i + 1], route=route_id)

    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    edge_text = []

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_text.append(f"Route: {edge[2]['route']}")

    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes:
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(f"Stop ID: {node}")

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        text=edge_text,
        mode='lines'
    )

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            size=10,
            color='#00bfff',
            line_width=2),
        text=node_text
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Stop-Route Graph',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False))
                    )

    # Save as HTML for viewing in a browser
    fig.write_html("stop_route_graph.html")
    print("Plot saved as 'stop_route_graph.html'. Open this file in a browser to view the interactive plot.")
    
    fig.show()

In [6]:
# Run the Knowledge Base creation
create_kb()  # Ensure this line is executed to populate route_to_stops

C:\Users\vikra\AppData\Local\Temp\ipykernel_3048\1676555569.py:34: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  fare_rules = df_fare_rules.set_index('route_id').T.to_dict()


In [7]:
import os

# Create the outputs directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), "outputs")
os.makedirs(output_dir, exist_ok=True)

def write_busiest_routes():
    busiest_routes = get_busiest_routes()
    file_path = os.path.join(output_dir, "busiest_routes.txt")
    with open(file_path, "w") as f:
        f.write("Top 5 Busiest Routes (based on trip counts):\n")
        for route_id, trip_count in busiest_routes:
            f.write(f"Route ID: {route_id}, Trip Count: {trip_count}\n")

def write_most_frequent_stops():
    most_frequent_stops = get_most_frequent_stops()
    file_path = os.path.join(output_dir, "most_frequent_stops.txt")
    with open(file_path, "w") as f:
        f.write("Top 5 Stops with the Most Frequent Trips:\n")
        for stop_id, trip_count in most_frequent_stops:
            f.write(f"Stop ID: {stop_id}, Trip Count: {trip_count}\n")

def write_top_5_busiest_stops():
    busiest_stops = get_top_5_busiest_stops()
    file_path = os.path.join(output_dir, "busiest_stops.txt")
    with open(file_path, "w") as f:
        f.write("Top 5 Busiest Stops:\n")
        for stop_id, route_count in busiest_stops:
            f.write(f"Stop ID: {stop_id}, Route Count: {route_count}\n")

def write_stops_with_one_direct_route():
    stops_with_one_route = get_stops_with_one_direct_route()
    file_path = os.path.join(output_dir, "stops_with_one_direct_route.txt")
    with open(file_path, "w") as f:
        f.write("Stops with Only One Direct Route:\n")
        for stop_id, route_id in stops_with_one_route:
            f.write(f"Stop ID: {stop_id}, Route ID: {route_id}\n")

# Run all functions to write outputs
write_busiest_routes()
write_most_frequent_stops()
write_top_5_busiest_stops()
write_stops_with_one_direct_route()

print("All outputs have been written to the 'outputs' directory.")

All outputs have been written to the 'outputs' directory.


In [8]:
# visualize_stop_route_graph_interactive(route_to_stops)

# REASONING

In [9]:
# Brute-Force Approach for finding direct routes (ignoring direction)
def direct_route_brute_force(start_stop, end_stop):
    """
    Find all valid routes between two stops using a brute-force method, ignoring direction.

    Args:
        start_stop (int): The ID of the starting stop.
        end_stop (int): The ID of the ending stop.

    Returns:
        list: A list of route IDs (int) that connect the two stops.
    """
    direct_routes = []

    for route_id, stops in route_to_stops.items():
        # Check if both stops are in the list of stops for this route
        if start_stop in stops and end_stop in stops:
            direct_routes.append(route_id)

    return direct_routes


In [10]:
## Testing Direct Route Brute Force

test_inputs = {
    "direct_route": [
        ((2573, 1177), [10001, 1117, 1407]),  # Input -> Expected output
        ((2001, 2005), [10001, 1151])
    ]
}

def check_output(expected, actual):
    """Function to compare expected and actual outputs."""
    if isinstance(expected, list) and isinstance(actual, list):
        return sorted(expected) == sorted(actual)  # Ensures order-independent comparison
    return expected == actual  # For non-list types

def test_direct_route_brute_force():
    for (start_stop, end_stop), expected_output in test_inputs["direct_route"]:
        actual_output = direct_route_brute_force(start_stop, end_stop)
        print(f"Test direct_route_brute_force ({start_stop}, {end_stop}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")
        
test_direct_route_brute_force()

Test direct_route_brute_force (2573, 1177):  Pass
Test direct_route_brute_force (2001, 2005):  Pass


In [11]:
# Initialize Datalog predicates for reasoning
pyDatalog.create_terms('RouteHasStop, DirectRoute, OptimalRoute, X, Y, Z, R, R1, R2')  
def initialize_datalog():
    """
    Initialize Datalog terms and predicates for reasoning about routes and stops.

    Returns:
        None
    """
    pyDatalog.clear()  # Clear previous terms
    print("Terms initialized: DirectRoute, RouteHasStop, OptimalRoute")  # Confirmation print

    # Define Datalog predicates
    DirectRoute(R, X, Y) <= (RouteHasStop(R, X) & RouteHasStop(R, Y) & (X != Y))

    # create_kb()  # Populate the knowledge base
    add_route_data(route_to_stops)  # Add route data to Datalog
    
# Adding route data to Datalog
def add_route_data(route_to_stops):
    """
    Add the route data to Datalog for reasoning.

    Args:
        route_to_stops (dict): A dictionary mapping route IDs to lists of stops.

    Returns:
        None
    """
    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            +RouteHasStop(route_id, stop_id)

# Function to query direct routes between two stops
def query_direct_routes(start, end):
    """
    Query for direct routes between two stops.

    Args:
        start (int): The ID of the starting stop.
        end (int): The ID of the ending stop.

    Returns:
        list: A sorted list of route IDs (str) connecting the two stops.
    """
    query_result = pyDatalog.ask("DirectRoute(R, {}, {})".format(start, end))
    if query_result is None:
        return []

    route_ids = [answer[0] for answer in query_result.answers]

    return sorted(route_ids)

In [12]:
## Testing FOL Query Direct Route

test_inputs = {
    "direct_route": [
        ((2573, 1177), [10001, 1117, 1407]),  # Input -> Expected output
        ((2001, 2005), [10001, 1151])
    ]
}

def check_output(expected, actual):
    """Function to compare expected and actual outputs."""
    if isinstance(expected, list) and isinstance(actual, list):
        return sorted(expected) == sorted(actual)  # Ensures order-independent comparison
    return expected == actual  # For non-list types

def test_query_direct_routes():
    for (start_stop, end_stop), expected_output in test_inputs["direct_route"]:
        actual_output = query_direct_routes(start_stop, end_stop)
        print(f"Test query_direct_routes ({start_stop}, {end_stop}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")
        
initialize_datalog()
test_query_direct_routes()

Terms initialized: DirectRoute, RouteHasStop, OptimalRoute
Test query_direct_routes (2573, 1177):  Pass
Test query_direct_routes (2001, 2005):  Pass


# Planning

In [13]:
pyDatalog.create_terms('RouteHasStop, DirectRoute, CanReachWithTransfer, X, Y, Z, R, R1, R2, Stops')

# Sample Data - You would replace this with your actual data
# Updated route_to_stops with additional data for testing
route_to_stops = {
    10153: [22540, 4686, 2573],
    1407: [4686, 2573],
    294: [951, 300, 340],
    712: [300, 340],
    1211: [951, 300, 340],        # Add additional routes for new test cases
    10453: [951, 300, 340],
    387: [951, 300, 340],
    49: [951, 300, 340],
    1571: [951, 300, 340],
    37: [951, 300, 340],
    1038: [951, 300, 340],
    10433: [951, 300, 340],
    121: [951, 300, 340]
}

# Add the data to PyDatalog globally
for route, stops in route_to_stops.items():
    for stop in stops:
        +RouteHasStop(route, stop)

# Define a direct route rule for stops on the same route
DirectRoute(X, Y, R) <= (RouteHasStop(R, X) & RouteHasStop(R, Y) & (X != Y))

# Define a rule for reaching an endpoint with a transfer at a via stop
CanReachWithTransfer(X, Y, Z, R1, R2) <= (DirectRoute(X, Z, R1) & DirectRoute(Z, Y, R2) & (R1 != R2))

# Forward chaining for optimal route planning
def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Perform forward chaining to find optimal routes considering transfers.

    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.

    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria, where each tuple contains:
              - route_id (int): The ID of the route.
              - stop_id (int): The ID of the stop.
    """
    paths = []

    # Find possible paths with a transfer at the via stop
    valid_paths = pyDatalog.ask(f"CanReachWithTransfer({start_stop_id}, {end_stop_id}, {stop_id_to_include}, R1, R2)")

    # Process results
    if valid_paths is not None:
        for answer in valid_paths.answers:
            route1, route2 = answer[0], answer[1]
            paths.append((route1, stop_id_to_include, route2))

    return paths

In [14]:
# Updated test inputs
test_inputs = {
    "forward_chaining": [
        ((22540, 2573, 4686, 1), [(10153, 4686, 1407)]),
        ((951, 340, 300, 1), [
            (1211, 300, 712), 
            (10453, 300, 712), 
            (387, 300, 712), 
            (49, 300, 712), 
            (1571, 300, 712), 
            (37, 300, 712), 
            (1038, 300, 712), 
            (10433, 300, 712), 
            (121, 300, 712)
        ])
    ]
}

# Check output function
def check_output(expected, actual):
    """Function to compare expected and actual outputs."""
    if isinstance(expected, list) and isinstance(actual, list):
        return sorted(expected) == sorted(actual)  # Ensures order-independent comparison
    return expected == actual  # For non-list types

# Test function for forward chaining
def test_forward_chaining():
    for (start_stop, end_stop, via_stop, max_transfers), expected_output in test_inputs["forward_chaining"]:
        actual_output = forward_chaining(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test forward_chaining ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

# Run tests
test_forward_chaining()

Test forward_chaining (22540, 2573, 4686, 1):  Pass
Test forward_chaining (951, 340, 300, 1):  Fail (Expected: [(1211, 300, 712), (10453, 300, 712), (387, 300, 712), (49, 300, 712), (1571, 300, 712), (37, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)], Got: [(1571, 300, 387), (1571, 300, 49), (1571, 300, 294), (1571, 300, 712), (1571, 300, 1038), (1571, 300, 10453), (1571, 300, 121), (1571, 300, 1211), (1571, 300, 10433), (1571, 300, 37), (10453, 300, 37), (10453, 300, 10433), (10453, 300, 1571), (10453, 300, 1211), (10453, 300, 121), (10453, 300, 1038), (10453, 300, 712), (10453, 300, 294), (10453, 300, 49), (10453, 300, 387), (294, 300, 37), (294, 300, 10433), (294, 300, 1571), (294, 300, 1211), (294, 300, 121), (294, 300, 10453), (294, 300, 1038), (294, 300, 712), (294, 300, 49), (294, 300, 387), (121, 300, 37), (121, 300, 10433), (121, 300, 1571), (121, 300, 1211), (121, 300, 10453), (121, 300, 1038), (121, 300, 712), (121, 300, 294), (121, 300, 49), (121, 300, 38